In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import os

import cv2
import shutil
import numpy as np
import pandas as pd
import albumentations as A

from PIL import Image, ImageDraw
#from ultralytics import YOLO

from utils.utils import list_file_r, copy_image_to_sub_dir
from utils.data_augmentation import batch_augment_with_bbox, batch_augment_with_mask, batch_invers_FDA

In [19]:
img = cv2.imread('datasets/CF_simulation/images/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.tif')
bbox = np.loadtxt('datasets/CF_simulation/bbox/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.txt', delimiter=' ')
bbox = np.concatenate((bbox, bbox[:,0:1]), axis=-1)[:,1:]
img_h, img_w, _ = img.shape
#print(img.shape)
r = np.random.default_rng()
resize_h, resize_w = r.random()*0.5, r.random()*0.9
resize_h, resize_w = int(resize_h*img_h), int(resize_w*img_w)
print(resize_h, resize_w)
augmentor = A.Compose([
    # Transform (random noises)
    A.Resize(resize_h, resize_w, p=1.0),
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Blur effect
    A.MotionBlur(p=0.2),
    A.Defocus(),
    # Crop
    #A.RandomCrop(width=450, height=450),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=90, p=0.5),
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
    #A.FDA(reference_images=[cv2.imread('datasets/CF_simulation/real_image/test/hdt5_0010.tif')],read_fn=lambda x:x,p=1)
    ]
    ,bbox_params=A.BboxParams(format='yolo', min_visibility=0.2))

augmented = augmentor(image=img,bboxes=bbox)
Image.fromarray(augmented['image']).show()

270 333


In [3]:
img = cv2.imread('datasets/CF_simulation/images/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.tif')
bbox = np.loadtxt('datasets/CF_simulation/bbox/01_line_Radius4_2_Density1500_1000_Width50_Reflection_0.txt', delimiter=' ')
bbox = np.concatenate((bbox, bbox[:,0:1]), axis=-1)[:,1:]

In [85]:
temp = np.random.randint(0, 256, [100, 100, 3], dtype=np.uint8)
target_temp = np.random.randint(0, 256, [100, 100, 3], dtype=np.uint8)
aug = A.Compose([A.FDA([target_temp], p=1, read_fn=lambda x: x)])
result = aug(image=temp)

In [7]:
augmented = augmentor(image=img,bboxes=bbox)

In [ ]:
augmentor = A.Compose([
    # Transform (random noises)
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2, fog_coef_lower=0.1, fog_coef_upper=0.4),
    A.transforms.RandomShadow(p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    # Blur effect
    A.MotionBlur(p=0.2),
    A.Defocus(p=0.2),
    # Crop
    A.BBoxSafeRandomCrop(p=0.5),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=45, p=0.75)
    #A.HorizontalFlip(p=0.1),
    #A.Rotate(),
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.2))

#batch_augment_with_bbox(images_path='datasets/CF_simulation/images/', bbox_path='datasets/CF_simulation/bbox', augmentor=augmentor, random_choice=1.0)


In [5]:
folder = 'D:/xie/Vision/mcp_yolo/datasets/CF_simulation/mask'
for fn in os.listdir(folder):
    if fn[3:7] == 'line':
        os.rename(os.path.join(folder, fn), os.path.join(folder, 'line', fn)) 


In [23]:
transforms = [
    # Blur effect
    A.MotionBlur(p=0.2, blur_limit=5),
    #A.Defocus(p=0.2, alias_blur=0.3),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=180, p=0.75),
    # Transform (random noises),
    A.transforms.Spatter(p=0.2),
    A.transforms.RandomFog(p=0.2, fog_coef_lower=0.02, fog_coef_upper=0.1),
    A.transforms.RandomShadow(p=0.2),
    # Photometric
    A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, p=0.2),
    #A.RandomBrightnessContrast(p=0.2),
    # Crop
    #A.BBoxSafeRandomCrop(p=0.6),
    A.RandomCrop(720, 720, p=0.2),
    A.RandomCrop(640, 640, p=0.2),
    A.RandomCrop(480, 480, p=0.2),
]


In [19]:
transforms = [
    # Blur effect
    #A.MotionBlur(p=1, blur_limit=9),
    #A.Defocus(p=1, alias_blur=0.3),
    # Dropout
    #   none
    # Geometric
    A.ShiftScaleRotate(rotate_limit=180, p=0.75),
    # Transform (random noises),
    #A.transforms.Spatter(p=0.2),
    #A.transforms.RandomFog(p=0.2, fog_coef_lower=0.02, fog_coef_upper=0.1),
    A.transforms.RandomShadow(p=1),
    # Photometric
    #A.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    #A.RandomBrightnessContrast(p=0.2),
    # Crop
    #A.BBoxSafeRandomCrop(p=0.5),
    A.RandomCrop(720, 720, p=0.1),
    A.RandomCrop(640, 640, p=0.1),
    A.RandomCrop(480, 480, p=0.1),
]


In [24]:
batch_augment_with_mask(images_path='datasets/CF_simulation/images/real', mask_path='datasets/CF_simulation/mask/real', target_folder='real',  random_choice=2000, transforms=transforms, FDA_targets=None)

In [16]:
batch_augment_with_mask(images_path='datasets/CF_simulation/images/line', mask_path='datasets/CF_simulation/mask/line', transforms=transforms, FDA_targets=None, random_choice=12000)
batch_augment_with_mask(images_path='datasets/CF_simulation/images/supp1', mask_path='datasets/CF_simulation/mask/supp1', transforms=transforms, FDA_targets=None, random_choice=3000)
batch_augment_with_mask(images_path='datasets/CF_simulation/images/supp2', mask_path='datasets/CF_simulation/mask/supp2', transforms=transforms, FDA_targets=None, random_choice=3000)
batch_augment_with_mask(images_path='datasets/CF_simulation/images/supp3', mask_path='datasets/CF_simulation/mask/supp3', transforms=transforms, FDA_targets=None, random_choice=3000)

In [16]:
for imp in list_file_r('datasets/CF_simulation/mask/background'):
    img=np.array(Image.open(imp))
    img = Image.fromarray(img>0)
    img.save(imp)

In [3]:
# apply inverseFDA
FDA_targets='datasets/CF_simulation/RDA_targets'
image_folder = 'datasets/CF_simulation/images/line/RDA_large/11'
batch_invers_FDA(src=image_folder, FDA_targets=FDA_targets, beta=0.4, p=0.5)

In [31]:
from utils.fft import inverseFDA
import PIL.Image as Image
import numpy as np

src_path = 'datasets/CF_simulation/images/supp2/augmented/01_line_Radius3_1_Density4000_500_Width20_Reflection_02.tif'
tgt_path = 'datasets/CF_simulation/real_image/test_gs/hdt_0146.tif'
tgt_path = 'datasets/CF_simulation/real_image/test_gs/image490.bmp'

src, tgt = Image.open(src_path), Image.open(tgt_path)
tgt = tgt.resize(src.size)
src, tgt = np.array(src), np.array(tgt)
img = Image.fromarray(inverseFDA(src, tgt, beta=0.8))
img.save('0.8.png')

320 320 33 33
